In [1]:
# access the postgresql db

import os
import psycopg2
from dotenv import load_dotenv

# Load environment variables
load_dotenv("../.env.local")

# Get database connection details from environment variables
DATABASE_URL = os.getenv("POSTGRES_URL")
# Get database connection details from URL
# Parse database URL into components
from urllib.parse import urlparse

url = urlparse(DATABASE_URL)
DB_USER = url.username 
DB_PASSWORD = url.password 
DB_HOST = url.hostname 
DB_PORT = url.port 
DB_NAME = url.path[1:] 

# Establish connection
def connect_to_db():
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        print("Successfully connected to PostgreSQL database")
        
        # Create a cursor
        cur = conn.cursor()
        
    except psycopg2.Error as e:
        print(f"Error connecting to PostgreSQL database: {e}")
    
    return conn, cur


In [2]:
conn, cur = connect_to_db()
try:
    # Execute query to get the most recent chat
    cur.execute("""
            SELECT * FROM public."Chat"
            ORDER BY "createdAt" DESC 
            LIMIT 1
        """)
        
    # Fetch the result
    last_chat = cur.fetchone()
    
    if last_chat:
        print("Last chat found:")
        print(last_chat)
    else:
        print("No chats found in database")
    
except psycopg2.Error as e:
    print(f"Error querying database: {e}")
    # Rollback the transaction to reset the connection state
    conn.rollback()
    # Close cursor and connection to clean up
    cur.close()
    conn.close()

Successfully connected to PostgreSQL database
Last chat found:
('485c654c-1bc4-40b8-ba71-e34d00800eea', datetime.datetime(2025, 4, 30, 3, 3, 0, 710000), '235049eb-15ee-4a2a-9d7f-6189e9bd956e', 'Dijkstras Algorithm Code Example', 'private', 'google-gemini-2.5-pro-preview-03-25')


In [3]:
import json
conn, cur = connect_to_db()
try:
    # Execute query to get messages for the last chat
    cur.execute("""
        SELECT * FROM public."Message_v2"
        WHERE "chatId" = %s
        ORDER BY "createdAt" ASC
    """, (last_chat[0],))  # Use the chat ID from last_chat tuple
    
    # Fetch all messages
    chat_messages = cur.fetchall()
    
    if chat_messages:
        print("Messages for this chat:")
        for message in chat_messages:
            print(json.dumps({
                "id": message[0],
                "chatId": message[1], 
                "role": message[2],
                "parts": message[3],
                "attachments": message[4],
                "createdAt": message[5].isoformat()                
            }, indent=2))
    else:
        print("No messages found for this chat")

except psycopg2.Error as e:
    print(f"Error querying messages: {e}")
    conn.rollback()
finally:
    # Clean up database resources
    cur.close()
    conn.close()


Successfully connected to PostgreSQL database
Messages for this chat:
{
  "id": "5354895a-32eb-41fe-93a8-108a9b9590a4",
  "chatId": "485c654c-1bc4-40b8-ba71-e34d00800eea",
  "role": "user",
  "parts": [
    {
      "text": "Write code to demonstrate djikstra's algorithm",
      "type": "text"
    }
  ],
  "attachments": [],
  "createdAt": "2025-04-30T03:03:00.722000"
}
{
  "id": "fff9f1e5-4cad-4968-be29-06075530a259",
  "chatId": "485c654c-1bc4-40b8-ba71-e34d00800eea",
  "role": "assistant",
  "parts": [
    {
      "type": "step-start"
    },
    {
      "type": "text",
      "text": "Okay, I can help with that! I'll create a Python implementation of Dijkstra's algorithm for you in the artifact panel."
    },
    {
      "type": "tool-invocation",
      "toolInvocation": {
        "state": "result",
        "step": 0,
        "toolCallId": "tool_606476291_0",
        "toolName": "createDocument",
        "args": {
          "title": "Dijkstra Algorithm Implementation",
          "kind